In [1]:
#Set 2

In [2]:
#P9

In [3]:
block = b"YELLOW SUBMARINE"

In [4]:
padding = (20 - len(block)%20)*'\x04'

In [5]:
block + padding

'YELLOW SUBMARINE\x04\x04\x04\x04'

In [6]:
#P10

In [7]:
import urllib2
from Crypto.Cipher import AES
import sys
sys.path.append( 'Set1' )
from P5 import repkeyxor

In [125]:
url = 'http://cryptopals.com/static/challenge-data/10.txt'

In [126]:
text = urllib2.urlopen(url).read()

In [127]:
text = ''.join(text.split()).decode('base64')

In [128]:
key = "YELLOW SUBMARINE"
iv = '\x00'*16

In [119]:
cipher = AES.new(key, AES.MODE_ECB)

CBC mode is pretty similar to ECB mode; the difference is described on wikipedia. The first 16-byte block gets decrypted by ECB, then XORed with the IV. To do this, I reused my repeating-key-xor code.
Following blocks are decoded by ECB, and then XORed with the previous encrypted block.

In [120]:
plaintext = ""
plaintext += repkeyxor(cipher.decrypt(text[:16]), iv).decode('hex')
for i in range(1,len(text)/16):
    plaintext += repkeyxor(cipher.decrypt(text[16*i:16*i + 16]), text[16*i - 16:16*i]).decode('hex')

In [121]:
print plaintext

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


In [131]:
#P11

In [15]:
import random

In [132]:
def generate_key():
    '''Generates a random 16-byte key'''
    key = ""
    for i in range(16):
        key += chr(random.randint(0,255))
    return key

In [133]:
def encryption_oracle(plaintext):
    '''This function generates a random key, and uses it to randomly create either an ECB or CBC
    cipher. It then uses this cipher to encrypt a string which is input, after adding padding.'''
    key = generate_key()
    left_padding = random.randint(5,10)*chr(0)
    right_padding = random.randint(5,10)*chr(0)
    if random.randint(0,1) == 0:
        cipher = AES.new(key, AES.MODE_ECB)
    else:
        cipher = AES.new(key, AES.MODE_CBC, generate_key())
    text = left_padding+plaintext+right_padding
    PKCS_padding = (16 - len(text)%16)*'\x04'
    return cipher.encrypt(text+PKCS_padding)

In [140]:
def detect_ECB(ciphertext):
    '''This returns "True" when given a string encrypted by ECB. It requires the string to be at least
    length 36. It may fail by chance in the very rare (1 in 8**16) case when different blocks are CBC
    encrypted into the same ciphertext.'''
    assert len(ciphertext) > 36
    block = ciphertext[16:32]
    return block == ciphertext[32:48]

In [144]:
for i in range(10):
    print detect_ECB(encryption_oracle(chr(0)*16*4))

True
False
False
True
False
False
False
True
True
True


In [20]:
#P12

In [21]:
global string
string = ''.join("""Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK""".split())
global key
key = generate_key()

In [145]:
def consistent_encryption_oracle(plaintext):
    '''Given a constant (global) key and encrypted string, this function takes the input and
    appends the string, then encrypts the result using ECB.'''
    right_padding = string.decode('base64')
    cipher = AES.new(key, AES.MODE_ECB)
    text = plaintext+right_padding
    PKCS_padding = (16 - len(text)%16)*'\x04'
    return cipher.encrypt(text+PKCS_padding)

In [146]:
input = "A"*16
consistent_encryption_oracle(input)[:16]

'\xe4@\x85\xfa+\xda3\xd8j\xa3@\xb4\xc1l\x05\xad'

In [147]:
consistent_encryption_oracle(input)

'\xe4@\x85\xfa+\xda3\xd8j\xa3@\xb4\xc1l\x05\xadr\xe3\x94i\x00\x13\xb7\xce\xe4\x00\xdd\xc7\xd0\xc4\xc9;\xf7Ij\xa4J\xa3\x8b\x1c\noC!\xb5qr\x1a\xf9\xea26\x1e\xc9\xf9\xa1ij\x1f\x1c$\xd7B6q\x8a\xb3~\x81\xc2P\xa4\xca\x97\xaa\xf9]\x88\xf5\xc3\xf5\xc3\xf5L]\x10\x9d\xa5\xe3\x803\xe4\xc5\xda\xf8\x9d\xaex\xbcE\xed\xb4\xdc\xf5}/\x1f\xe9\xaf\xde\x93\xf4\xccho\xc0`\x94N\xb8\xc5B\xb2\xa1\xbe\xac\x11`\xccYN\xde\xf8\xf3^\xa1\x8a\xa9\xb3\xe0\xbb\xf4Ud&\x04\xc2\xa8\x9b\xc4B[jz~E\x81\x02\xe7}'

In [148]:
input = "A"*15
consistent_encryption_oracle(input)[:16]

';\x1e\t\xf0\xd9\x95\xf2\x82L\xc8.\x12\xdd\xa5\xf2\x91'

In [149]:
for i in range(256):
    if consistent_encryption_oracle("A"*15+chr(i))[:16] == consistent_encryption_oracle("A"*15)[:16]:
        print i

82


In [150]:
chr(82)

'R'

If the input is 15 characters long, the first byte of the secret string will be encrypted together with my 15 character input.

In [28]:
for i in range(256):
    if consistent_encryption_oracle("A"*14+"R"+chr(i))[:16] == consistent_encryption_oracle("A"*14)[:16]:
        print i

111


In [151]:
inputl = "A"*16
inputr = "A"*16
for b in range(16):
    inputl = inputl[1:]
    inputr = inputr[1:]
    for i in range(256):
        if consistent_encryption_oracle(inputl+chr(i))[:16] == consistent_encryption_oracle(inputr)[:16]:
            char = chr(i)
            print char
            inputl += char
            break

R
o
l
l
i
n
'
 
i
n
 
m
y
 
5
.


By decreasing the length of my input, I can guess at the secret character within 256 tries. Now, I can guess at the contents of the string in linear time, rather than exponential.

Next, we have to apply this to the following blocks. This is a little tricky; instead of an arbitrary 15-byte (or less) input, we have to use the previous block that we decrypted.

In [102]:
plaintext = ""
previous_block = "A"*16
n = 0
while len(plaintext) < len(consistent_encryption_oracle("")):
    inputl = previous_block
    inputr = "A"*16
    for b in range(16):
        inputl = inputl[1:]
        inputr = inputr[1:]
        for i in range(256):
            if consistent_encryption_oracle(inputl+chr(i))[:16] == consistent_encryption_oracle(inputr)[16*n:16*n+16]:
                char = chr(i)
                inputl += char
                break
    previous_block = inputl+""
    plaintext += inputl
    n += 1

print plaintext

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



Let's see if we were successful:

In [104]:
print string.decode('base64')

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



In [105]:
#P13